# Inconsistent Feature Preprocessing

Man muss immer das identische Feature Preprocessing auf allen Daten (Train-Set, Val-Set, Test-Set, in Produktion) anwenden.

## Von Hand

Hier wird gezeigt, wie man es richtig macht.
Die kritische Stelle, die oft vergessen bzw. falsch implementiert wird, ist mit einem Kommentar gekennzeichnet.

In [3]:
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [4]:
df = pd.read_csv('data/fish.csv')[['Width', 'Weight']].rename(columns={
    'Width': 'width (cm)',
    'Weight': 'weight (g)'
})

X = df[['width (cm)']]
y = df['weight (g)']

In [5]:

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42)

ss = StandardScaler()
ss.fit(X_train)  # "Lernphase": Berechnet Mean und Standardabweichung
X_train_std = ss.transform(X_train)

lr = Ridge()
lr.fit(X_train_std, y_train)

"""
Kritische Stelle:
1. We use the same StandardScaler as for X_train.
2. We do NOT fit the StandardScaler on the validation data!
"""
X_val_std = ss.transform(X_val)
y_val_hat = lr.predict(X_val_std)

## Pipelines

`sklearn` hat das Konzept der `Pipeline`, um diese mehrschrittigen Prozesse zentral in einer Pipeline festzulegen.
Das Anwenden von `fit` und `transform` wird innerhalb der Pipeline dann richtig (analog zu oben) behandelt.
Mit einer Pipeline kann man es dann gar nicht mehr falsch machen!

In [6]:
model = Pipeline([
    ('ss', StandardScaler()),
    ('lr', Ridge()),
])
model.fit(X_train, y_train)         # Fits StandardScaler and Ridge, analog to this: "ss.fit(X_train); ridge.fit(ss.transform(X_train))"
y_val_hat = model.predict(X_val)    # Applies StandardScaler and Ridge to X_test, analog to this: "ridge.predict(ss.transform(X_test))"

Pipelines sind nützlich und führen zu verständlicherem Code.